<a href="https://colab.research.google.com/github/SergheiMihailov/ml-project-cassava/blob/tuning-hp/tuning_hp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install -q -U keras-tuner

In [1]:
# Imports
import gdown
import os
import json
import csv   
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import scipy.misc
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow.keras.layers.experimental.preprocessing as keras_preproc
import kerastuner as kt
from pprint import pprint


In [29]:
# Download provided dataset
!wget files.brainfriz.com/train_images.zip # secondary link for images
!unzip -qq -o train_images.zip
!gdown --id "1xbEVK_NigW_5ngwKMHvuOTehYhT2v2WF" # labels
!gdown --id "1SvI9dN2_25c2OlevwK4TjmzBNysjE_PO" # label mapping

--2021-03-14 14:40:34--  http://files.brainfriz.com/train_images.zip
Resolving files.brainfriz.com (files.brainfriz.com)... 138.201.201.196
Connecting to files.brainfriz.com (files.brainfriz.com)|138.201.201.196|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.brainfriz.com/train_images.zip [following]
--2021-03-14 14:40:34--  https://files.brainfriz.com/train_images.zip
Connecting to files.brainfriz.com (files.brainfriz.com)|138.201.201.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2569658627 (2.4G) [application/zip]
Saving to: ‘train_images.zip.1’

train_images.zip.1  100%[===================>]   2.39G  20.4MB/s    in 2m 4s   

2021-03-14 14:42:39 (19.8 MB/s) - ‘train_images.zip.1’ saved [2569658627/2569658627]

Downloading...
From: https://drive.google.com/uc?id=1xbEVK_NigW_5ngwKMHvuOTehYhT2v2WF
To: /content/train.csv
100% 358k/358k [00:00<00:00, 56.8MB/s]
Downloading...
From: https://drive.google

In [2]:
IMG_SIZE = 512

def augment_image(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
  img4d = tf.expand_dims(img, 0)
  data_augmentation = tf.keras.Sequential([
    keras_preproc.Resizing(IMG_SIZE, IMG_SIZE),
    keras_preproc.RandomRotation(0.2),
    keras_preproc.RandomZoom((0,-0.3)),
  ])

  aug_img_arr = data_augmentation(img4d)

  aug_img = Image.fromarray(aug_img_arr.numpy()[0].astype(np.uint8))
    
  return aug_img

def add_train_datapoint_cassava(image, image_id, label, train_images_dir_path, train_csv_path):
    datapoint = dict({
        'image_id': image_id,
        'label': label,
    })
    
    if not os.path.exists(train_images_dir_path):
        os.makedirs(train_images_dir_path)
      
    image.save(train_images_dir_path + str(image_id)) # save
  
    with open(train_csv_path, 'a') as f:
      writer = csv.DictWriter(f, ['image_id', 'label'])
      writer.writerow(datapoint)


In [ ]:
!zip -r orig_and_aug_train_images.zip orig_and_aug_train_images

In [ ]:
from google.colab import files
# files.download('orig_and_aug.csv')
files.download('orig_and_aug_train_images.zip') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
N_CLASSES = 5
IMG_SIZE = 512
SIZE = (IMG_SIZE,IMG_SIZE)
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
CLASSES = 5
BATCH_SIZE = 16
N_CV_SPLITS = 3

def getEfficientNetB0():
  return [
      # architecture
      tf.keras.applications.EfficientNetB0(
        include_top=True, weights=None, input_tensor=None,
        input_shape=INPUT_SHAPE, pooling=None, classes=N_CLASSES,
        classifier_activation='softmax', drop_connect_rate=0.4
      ),
      # preprocess_input
      tf.keras.applications.efficientnet.preprocess_input
  ]

def getResNet50V2(): 
  return [
      # architecture 
      tf.keras.applications.ResNet50V2(
        include_top=True, weights=None, input_tensor=None,
        input_shape=INPUT_SHAPE, pooling=None, classes=N_CLASSES,
        classifier_activation='softmax'
      ),
      # preprocess_input
      tf.keras.applications.resnet_v2.preprocess_input
  ]

def getMobileNetV3Small(): 
  return [
      # architecture
      tf.keras.applications.MobileNetV3Small(
        input_shape=INPUT_SHAPE, alpha=1, minimalistic=True, include_top=True,
        weights=None, input_tensor=None, classes=5, pooling='avg',
        dropout_rate=0, classifier_activation='softmax'
      ),
      # preprocess_input:
      tf.keras.applications.mobilenet_v3.preprocess_input
  ]



In [ ]:
# Train with cross-validation
data = pd.read_csv('train.csv')
f = open('label_num_to_disease_map.json')
real_labels = json.load(f)
real_labels = {int(k):v for k,v in real_labels.items()}
data['class_name'] = data.label.map(real_labels)

train_path = 'train_images/'

imageDataGenerator = ImageDataGenerator()

def model_builder(hp):
  architecture, preprocess_input = getMobileNetV3Small()

  input_layer = preprocess_input(tf.keras.layers.Input(shape=INPUT_SHAPE))

  model = tf.keras.Model(input_layer, architecture(input_layer))
  
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])
  hp_label_smoothing = hp.Choice('label_smoothing', values=[1e-1, 1e-2, 1e-3, 1e-4])
  # hp_alpha = hp.Choice('alpha', values=[0.1, 0.2, 0.5, 1.0, 1.1, 1.2, 1.5])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.BinaryCrossentropy(label_smoothing=hp_label_smoothing), 
                metrics=['accuracy'])

  return model

tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=10,
                     factor=3,
                     directory='hyperparams',
                     overwrite=True
                     )

stop_early = tf.keras.callbacks.EarlyStopping(
                monitor="val_loss",
                patience=2,
                verbose=2,
                mode="min",
            )
best_hps = None


kfold = StratifiedKFold(n_splits = N_CV_SPLITS)
cv_index = 0

for train_indices, val_indices in kfold.split(data['image_id'], data['label']):
  print('Training on cross-validation split '+str(cv_index))
  train_ds = data.iloc[train_indices]
  val_ds = data.iloc[val_indices]

  train_set = imageDataGenerator.flow_from_dataframe(train_ds,
                                  subset='training',
                                  directory = train_path,
                                  x_col = 'image_id',
                                  y_col = 'class_name',
                                  target_size = SIZE,
                                  color_mode="rgb",
                                  class_mode = 'categorical',
                                  batch_size = BATCH_SIZE)

  val_set = imageDataGenerator.flow_from_dataframe(val_ds,
                                  directory = train_path,
                                  x_col = 'image_id',
                                  y_col = 'class_name',
                                  target_size = SIZE,
                                  color_mode="rgb",
                                  class_mode = 'categorical',
                                  batch_size = BATCH_SIZE)
  
  
  if cv_index == 0:
    # Tune hyperparameters on first cross-validation (refactor later to use saved hps)
    tuner.search(train_set, validation_data=val_set, epochs=10, callbacks=[stop_early])

    # Get the optimal hyperparameters
    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
    print(f"""
    The hyperparameter search is complete. The optimal learning rate for the optimizer
    is {best_hps.get('learning_rate')}.
    """)
      
  else:
    model = tuner.hypermodel.build(best_hps)
    history = model.fit(
          train_set,
          steps_per_epoch=train_set.n // 32,
          epochs=30,
          validation_data=val_set,
          validation_steps=val_set.n // 32,
          verbose=2
    )
    cv_index += 1
    val_acc_per_epoch = history.history['val_accuracy']
    best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
    print('Best epoch: %d' % (best_epoch,))

Trial 2 Complete [00h 08m 48s]
val_loss: 0.46094968914985657

Best val_loss So Far: 0.43705713748931885
Total elapsed time: 00h 17m 38s

Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
learning_rate     |0.0001            |0.001             
label_smoothing   |0.01              |0.001             
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |2                 |2                 
tuner/round       |0                 |0                 

Epoch 1/2
892/892 [==============================] - ETA: 0s - loss: 0.3839 - accuracy: 0.6040

KeyboardInterrupt: ignored